In [1]:
import torch
import torchvision
from torchvision import datasets, models, transforms
from PIL import Image

import numpy as np
import os

In [2]:
def transform_image(path):

    image_transform = transforms.Compose([transforms.Resize((224,224)),
                                          transforms.ToTensor()])
    
    image = Image.open(path)
    image = image_transform(image)

    return image

In [3]:
def _load_data(batch_size,num_workers) -> torch.utils.data.DataLoader:
    """ Returns training and test data """
    
    ## defining the folders
    train_data_raw = os.path.abspath('../data/train/')
    test_data_raw = os.path.abspath('../data/test/')
    classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
    
    ## transformations
    data_transform = transforms.Compose([transforms.Resize((224,224)),
                                        transforms.ToTensor()])
    ## selecting the folders
    train_data = datasets.ImageFolder(train_data_raw, transform=data_transform)
    test_data = datasets.ImageFolder(test_data_raw, transform=data_transform)
    
    ## loading the data
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                            num_workers=num_workers, shuffle=True)
    
    return train_loader, test_loader

def _build_pretrained_model(cuda=True) -> torchvision.models.vgg16:
    """ Returns a built model """

    model = models.vgg16(pretrained=True)
    
    for parameter in model.features.parameters():
        parameter.requires_grad = False
        
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=5, bias=True)
    model.classifier.add_module = torch.nn.Softmax(5)
    
    if cuda:
        model.cuda()
    
    return model

def train_model(epochs:int) -> torchvision.models.vgg16:
    """ Train a model """
    
    train_loss = 0.0
    model = _build_pretrained_model()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    
    train,test = _load_data(32,0)
    
    for epoch in range(epochs):
        for data in train:
            
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            
            train_loss += loss.item()*inputs.size(0)
            
            optimizer.step()
            optimizer.zero_grad()

    train_loss = train_loss/len(train.dataset)
    
    return model, train, test, train_loss


def test_model(model:torchvision.models.vgg16, test:torch.utils.data.DataLoader):
    """ Eval a model using test data """
    
    criterion = torch.nn.CrossEntropyLoss()

    test_loss = 0.0
    correct, total = 0,0
    predictions = []
    model.eval()
    
    for data in test:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        loss = criterion(outputs,labels)
        test_loss += loss.item()*inputs.size(0)

        predictions.append(predicted)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_loss = test_loss/len(test.dataset)
    acc = (100*correct/total)

    return round(acc,2), test_loss


def save_model(model:torchvision.models.vgg16, acc:int):
    torch.save(model,f'TrainModel/models/model{acc}')

In [23]:
train_model(2)

RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 5.94 GiB total capacity; 3.29 GiB already allocated; 230.94 MiB free; 3.31 GiB reserved in total by PyTorch)

In [7]:
def _load_data(batch_size,num_workers) -> torch.utils.data.DataLoader:
    """ Returns training and test data """
    
    ## defining the folders
    train_data_raw = os.path.abspath('../data/train/')
    test_data_raw = os.path.abspath('../data/test/')
    classes = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']
    
    ## transformations
    data_transform = transforms.Compose([transforms.Resize((224,224)),
                                         transforms.ToTensor()])
    ## selecting the folders
    train_data = datasets.ImageFolder(train_data_raw, transform=data_transform)
    test_data = datasets.ImageFolder(test_data_raw, transform=data_transform)
    
    ## loading the data
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                               num_workers=num_workers, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                              num_workers=num_workers, shuffle=True)
    
    return train_loader, test_loader

def _build_pretrained_model(cuda=True) -> torchvision.models.vgg16:
    model = models.vgg16(pretrained=True)
    
    for parameter in model.features.parameters():
        parameter.requires_grad = False
        
    model.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=5, bias=True)
    
    if cuda:
        model.cuda()
    
    return model

def train(epochs:int) -> torchvision.models.vgg16:
    
    model = _build_pretrained_model()
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
    
    train,test = _load_data(32,0)
    
    for epoch in range(epochs):
        for data in train:
            
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            outputs = model(inputs)
            loss = criterion(outputs,labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        print(f'Epoch', epoch, end=' ')
    
    return model, train, test


def test_model(model:torchvision.models.vgg16, test:torch.utils.data.DataLoader):
    
    correct, total = 0,0
    predictions = []
    model.eval()
    
    for data in test:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        
        predictions.append(predicted)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'ACC: ',(100*correct/total))

def main():
    if torch.cuda.is_available():
        print('CUDA AVAILABLE')
    else:
        print('USING CPU')

In [3]:
model, train, test = train(15)

Epoch  0 Epoch  1 Epoch  2 Epoch  3 Epoch  4 Epoch  5 Epoch  6 Epoch  7 Epoch  8 Epoch  9 Epoch  10 Epoch  11 Epoch  12 Epoch  13 Epoch  14 

In [4]:
test_model(model,test)

ACC:  80.74074074074075


In [8]:
model, train, test = train(15)

Epoch 0 Epoch 1 Epoch 2 Epoch 3 Epoch 4 Epoch 5 Epoch 6 Epoch 7 Epoch 8 Epoch 9 Epoch 10 Epoch 11 Epoch 12 Epoch 13 Epoch 14 

In [9]:
test_model(model,test)

ACC:  82.96296296296296
